This nb will be used to qucikly access CR+SH and ScopusAS API loops to add new articles as necessary

### 1- CR + SH:

In [ ]:
import pandas as pd
import numpy as np
import pickle

from random import choice

from useful_functions import *

# from crossref.restful import Works, Etiquette
# from scihub_blastoise import SciHub_HydroPump


In [ ]:
# pickle here:

with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

#### 1.1 - Pivots & Required_Count

In [ ]:
with open("7b_variables/q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

In [ ]:
current_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")

In [ ]:
missing_pivot = (q1_a_ss - current_pivot).applymap(lambda x: x if x>0 else 0)

In [ ]:
# Creating empty columns:

q1_jrnl_df["Required_Count"] = q1_jrnl_df.Sbj_Area.map(dict)
q1_jrnl_df["Required_Count"] = q1_jrnl_df.Required_Count.map(lambda x: dict.fromkeys(x,int(0)))

q1_jrnl_df["SA_list"] = q1_jrnl_df.Required_Count.map(lambda x: list(x.keys()))

q1_jrnl_df["Required_Count_Sum"] = 0

In [ ]:
# Distributing missing articles to journals:

multip = 1.5

for s_a, row in missing_pivot.iterrows(): #iterate over rows
    # print(s_a)
    for year, value in row.items():
        # print(year)
        if value == 0:
            pass
        else:
            for i in range(int(value*multip)):
                # select one: 
                sampl_index = q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.CR_Count > (q1_jrnl_df.Required_Count_Sum + q1_jrnl_df.Current_Count)) & (q1_jrnl_df.SA_list.map(lambda x: s_a in x)) & (q1_jrnl_df.Year == year)].sample(1).index.item()
        
                q1_jrnl_df.loc[sampl_index,"Required_Count"][s_a] += 1
                q1_jrnl_df.loc[sampl_index,"Required_Count_Sum"] += 1

#### 1.2 - Required_df & all_dois & CR + SH Loops

In [ ]:
# DOI list to be skipped in the loop:

all_dois = set(q1_date_df.index.to_list() + remainder_df.index.to_list())

with open("all_dois","wb") as p:
    pickle.dump(all_dois, p)

In [ ]:
# Filtered version of jrnl_df to be used in the loop:

required_df = q1_jrnl_df[q1_jrnl_df.Required_Count_Sum > 0]

with open("required_df","wb") as p:
    pickle.dump(required_df, p)

In [ ]:

sh_h = SciHub_HydroPump()

mail = 'sduzenli@yandex.com.tr'

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', mail)
ogi_works = Works(etiquette=my_etiquette)


partial_df = required_df[["ISSN","Year", "Required_Count_Sum","CR_Count"]]

partial_df.loc[:,"ISSN"] = partial_df.ISSN.map(list)
pr_dict = partial_df.to_dict("index")

new_run_cr_results = list()
new_run_sh_results = list() 


useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'container-title', 'issued', 'ISSN', 'subject']

for key in pr_dict:
    issn = pr_dict[key]["ISSN"]
    year = pr_dict[key]["Year"]

    rc = pr_dict[key]["Required_Count_Sum"]
    crc = pr_dict[key]["CR_Count"]
    min_rc = min(rc+3, crc)

    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(int(min_rc)).select(useful_cols)]
    loop_meta = [dict(item, JRNL_ID=key) for item in loop_meta if item["DOI"] not in all_dois]
    new_run_cr_results.extend(loop_meta)

    for artcl in loop_meta:
        doi = artcl["DOI"]
        pdf_url, date, kw = sh_h.get_dates_kws(doi)
        new_run_sh_results.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))


In [ ]:
with open("new_req_cr_results","wb") as fp:
   pickle.dump(new_run_cr_results, fp)

with open("new_req_sh_results","wb") as fp:
   pickle.dump(new_run_sh_results, fp)


#### 1.3 - Run Results' Analysis

In [ ]:
with open("new_req_cr_results_final", "rb") as fp:
    new_run_cr_results_final = pickle.load(fp)

with open("new_req_sh_results_final", "rb") as fp:
    new_run_sh_results_final = pickle.load(fp)


In [ ]:
# Create df (CR + SH):
new_run_df = pd.DataFrame(new_run_cr_results_final).merge(pd.DataFrame(new_run_sh_results_final), how="inner", on="DOI")

# to be added to date_df:
new_run_succ = new_run_df[new_run_df.Dates.map(lambda x: type(x) == list)]
new_run_succ = new_run_succ[new_run_succ.Dates.map(lambda x: type(x[0]) == str)]

# to be added to remainder_df:
new_run_rem = new_run_df[~new_run_df.DOI.isin(new_run_succ.index.to_list())]


In [ ]:
del new_run_cr_results_final
del new_run_sh_results_final

In [ ]:
# Format engineering on the new_run_succ dataset:

new_run_succ.set_index("DOI", drop=True, inplace=True)
new_run_succ.loc[:,"published"] = new_run_succ.apply(earlier_date_part, axis=1)
new_run_succ = new_run_succ[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]
new_run_succ.loc[:,"JRNL_YEAR"] = new_run_succ.JRNL_ID.map(q1_jrnl_df.Year.to_dict())

In [ ]:
# Assigning subj. areas to successful articles:

for id, row in new_run_succ.iterrows():
    jrnl_id = row["JRNL_ID"]
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count"]
    

    if sum(list(rc_dict.values())) == 0:
        new_run_succ.loc[id,"Chosen_SA"] = choice(list(rc_dict.keys()))

    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        q1_jrnl_df.loc[jrnl_id,"Required_Count"][chosen_sa] -= 1
        new_run_succ.loc[id,"Chosen_SA"] = chosen_sa


In [ ]:
# Small modifications on jrnl_df & saving:

q1_jrnl_df["New_Additions"] = new_run_succ.groupby("JRNL_ID")["title"].count()
q1_jrnl_df.New_Additions.fillna(0, inplace=True)
q1_jrnl_df["New_Additions"] = q1_jrnl_df.New_Additions.astype(int)

with open("q1_jrnl_df","wb") as p:
        pickle.dump(q1_jrnl_df, p)

with open("q1_jrnl_df_copy","wb") as p:
        pickle.dump(q1_jrnl_df, p)

In [ ]:
# Adding new articles to date_df & saving:

q1_date_df = pd.concat([q1_date_df,new_run_succ],verify_integrity=True)

with open("q1_date_df","wb") as p:
        pickle.dump(q1_date_df, p)

with open("q1_date_df_copy","wb") as p:
        pickle.dump(q1_date_df, p)

In [ ]:
# Adding new articles to remainder_df & saving:

with open("q1_remainder_df", "rb") as fp:
    q1_remainder_df = pickle.load(fp)

q1_remainder_df = pd.concat([q1_remainder_df, new_run_rem.set_index("DOI")], verify_integrity=True)

with open("q1_remainder_df","wb") as p:
        pickle.dump(q1_remainder_df, p)

In [ ]:
# Saving datasets to the run folder:

# with open("7e_variables/last_5k_run_Sum/k5k_new_df","wb") as p:
#         pickle.dump(k5k_new_df, p)

# with open("7e_variables/last_5k_run_Sum/new_run_succ","wb") as p:
#         pickle.dump(new_run_succ, p)

# with open("7e_variables/last_5k_run_Sum/k5k_new_rem_df","wb") as p:
#         pickle.dump(k5k_new_rem_df, p)

In [ ]:
# Deleting unnec. variables:

del q1_remainder_df
del new_run_df
del new_run_succ
del new_run_rem

### 2- Scopus Author Search API:

#### 2.1 - author_df & first_auhtor_df

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import pickle

from useful_functions import *

from datetime import datetime
from time import sleep

from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elssearch import ElsSearch


In [ ]:
with open("7e_variables/new_run_succ", "rb") as fp:
    new_run_succ = pickle.load(fp)

In [ ]:
# Creating new_author_df from new_suc_df

new_author_df = new_run_succ.dropna(subset= "author").reset_index().loc[:,["DOI", "JRNL_ID", "author"]].explode("author").reset_index(drop=True)
new_aut_df = pd.DataFrame(list(new_author_df['author']))

# Combining the two datasets:

new_author_df = pd.concat([new_author_df, new_aut_df], axis=1)
new_author_df.drop(["suffix", "name", "author"], axis=1, inplace=True)

# Drop empty given & family:

new_author_df.dropna(subset=["family", "given"], inplace=True)

In [ ]:
# strip():
new_author_df.given = new_author_df.given.str.strip()
new_author_df.family = new_author_df.family.str.strip()

# Remove multiple spaces:
new_author_df.given = new_author_df.given.map(lambda x: re.sub(' +', ' ', x))
new_author_df.family = new_author_df.family.map(lambda x: re.sub(' +', ' ', x))


# Case 1: A. C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x)))].given.map(lambda x: x +".")
# Case 2: A.C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x)))].given.map(lambda x: x.replace(".",". ") +".")
# Case 3: AC
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x)))].given.map(lambda x: x[0] + ". " + x[1] + ".")
# Case 4: A C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x)))].given.map(lambda x: x.replace(" ", ". ") + ".")
# Case 5: A.C.
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x))), "new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x)))].given.map(lambda x: x.replace(".", ". ").strip())

# Single letter names:
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x)))].given.map(lambda x: x + ".")



In [ ]:
# Name Simplifier:

new_author_df.loc[:,"new_given"] = new_author_df.given.map(name_simplifier)
new_author_df.loc[:,"new_family"] = new_author_df.family.map(name_simplifier)

# After process stripping:

new_author_df.given = new_author_df.given.str.strip()
new_author_df.family = new_author_df.family.str.strip()

new_author_df.new_given = new_author_df.new_given.str.strip()
new_author_df.new_family = new_author_df.new_family.str.strip()



In [ ]:
# Adding Subj. Area column to the new_author_df:

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

new_author_df = pd.merge(left=new_author_df, right=q1_jrnl_df["SA_list"], how="left", left_on="JRNL_ID", right_index=True)

In [ ]:
# Creating new_first_author_df


new_first_author_df = new_author_df[new_author_df.sequence == "first"].groupby(["new_given", "new_family"])["SA_list"].apply(list)
new_first_author_df = new_first_author_df.to_frame().merge(new_author_df[new_author_df.sequence == "first"].groupby(["new_given", "new_family"])["DOI"].apply(list), how= "inner", right_index=True, left_index=True)
new_first_author_df.reset_index(inplace=True)

In [ ]:
# Checking for any problamatic names:

new_first_author_df[new_first_author_df.new_given.map(lambda x: bool(re.match("\W",x)))]
new_first_author_df[new_first_author_df.new_family.map(lambda x: bool(re.match("\W",x)))]

# if found:
new_first_author_df.drop(24266, inplace=True)

In [ ]:
# 1- Flattening SA_list col:

new_first_author_df.loc[:,"SA_list"] = new_first_author_df.SA_list.map(flatten)

# 2- Creating an Author ID col:

new_first_author_df.reset_index(drop=True, inplace=True)
new_first_author_df.reset_index(drop=False,inplace=True)
new_first_author_df.loc[:,"index"] = new_first_author_df.index.map(lambda x: "aut_" + str(x))
new_first_author_df.set_index("index", inplace=True)

In [ ]:
with open("new_first_author_df","wb") as p:
    pickle.dump(new_first_author_df, p)

with open("new_author_df","wb") as p:
    pickle.dump(new_author_df, p)

#### 2.2 - Scopus loop

In [ ]:
new_author_input_dict = new_first_author_df[["new_given","new_family","SA_list"]].to_dict("index")

In [ ]:
# Full Loop - Improved:

new_author_out_list= list()

avail_api_keys_list = ["8707db153e4b9672fa6df25b03a5f747","2dab4694b4347fa574d159bb97484fc4","6af3d2c09eb08ec6e12f0cead9a1f5bb","3872e798bf48fa28af583b9ebef5deb6","1473c31dbcdb425f9cdaf75c673279d3"]

used_api_keys_list = list()

current_key = avail_api_keys_list.pop(0)
client = ElsClient(current_key)

print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in new_author_input_dict:
    given_in = new_author_input_dict[auth_id]["new_given"]
    family_in = new_author_input_dict[auth_id]["new_family"]
    sa_in = new_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
        cnt_check = True

    except Exception as e:
        err_no = re.search("\d+",str(e))[0]

        print(f"Encountered Error: {err_no}!")
        print(f'Time = {datetime.now().strftime("%H:%M:%S")}')

        with open("new_author_out_list_loop_save","wb") as p:
                pickle.dump(new_author_out_list, p)
        print("Loop save complete!")

        if "400" in str(e):
            # query error, need to skip
            print(f"Skipping author: {given_in} {family_in} - {auth_id}")
            cnt_check = False

        elif "401" in str(e):
            # VPN error, sleep try again
            sleep(60)
            print("Slept for 60 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                print("Got an error again & breaking the loop ! :(")
                break
            
        elif "429" in str(e):
            # change API key 
            while len(avail_api_keys_list) >0:
                used_api_keys_list.append(current_key)
                current_key = avail_api_keys_list.pop(0)
                client = ElsClient(current_key)
                print(f"Changed API key, now using key: {current_key}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("New key is OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "429":
                        print("New key is also finished!")
                    else:
                        print(f"Skipping author: {given_in} {family_in} - {auth_id}")
                        cnt_check = False
                        break

            if len(avail_api_keys_list) == 0:
                print("No available API keys left & breaking the loop! :(")
                break

        elif "500" in str(e):
            sleep(30)
            print("Slept for 30 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                cnt_check = False
            pass
        
        elif "10060" in str(e):
            cnt_check = False
            for i in range(10):
                sleep(30)
                print("Slept for 30 & trying again!")
                print(f"Trial number: {i+1}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("It is now OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "10060":
                        print(f"It is still {err_no}!")
                    else:
                        print(f"Another error: {err_no}")
            
            if not cnt_check:
                print(f"Couldn't fix {err_no} & breaking the loop! :(")
                break

        
        else:
            print("Unknown error & breaking the loop! :(")
            break



    if cnt_check:
        country_out = list()
        given_out = list()
        family_out = list()
        doc_count_out = list()

        for auth in auth_srch.results:
            # Country:
            try:
                country_out.append(auth['affiliation-current']['affiliation-country'])
            except:
                pass

            # Given:
            try:
                given_out.append(auth['preferred-name']['given-name'])
            except:
                pass

            # Family:
            try:
                family_out.append(auth['preferred-name']['surname'])
            except:
                pass

            # Doc Count:
            try:
                doc_count_out.append(auth['document-count'])
            except:
                pass

        author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)
        new_author_out_list.append(author_out_dict)




In [ ]:
# To check progress in case of an error:

pd.DataFrame(new_author_out_list)

In [ ]:
with open("new_author_out_list","wb") as p:
    pickle.dump(new_author_out_list, p)

#### 2.3 - Combining Scopus results datasets:

In [ ]:
# Loading if necessary:

with open("location", "rb") as fp:
    new_author_df = pickle.load(fp)

with open("location", "rb") as fp:
    new_first_author_df = pickle.load(fp)

with open("location, "rb") as fp:
    new_author_out_list = pickle.load(fp)

In [ ]:
new_first_author_df = new_first_author_df.merge(pd.DataFrame(new_author_out_list).set_index("author_id"), how="left", left_index=True, right_index=True)

# Then combine with the main first_author_df as necessary:
all_first_author_df = pd.concat([all_first_author_df,new_first_author_df], ignore_index=True, verify_integrity=True)